# **Libraries**





In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from pandas import datetime
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import plotly.express as px
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots

# Background & Problem Statement

ROSSMANN is a germany large drug store chain that operates accross the Europe, the objective of this study is to predict 6 weeks of daily sales for 1,115 ROSSMANN drug stores located across Germany, as reliable sales forecasts enable store managers to increase the overall productivity and profitability of the retail business and improve their customer satisfaction.  

However, the challenges in this sales forecasting problem are to take into account various types of factors and to deal with missing data from historical records.

---



# 'Stores' Dataset's Columns and Description

1.   **Store**:  the anonymised store number
2.   **StoreType**: 4 different store models: a, b, c, d
3.   **Assortment**: an assortment level: a = basic, b = extra, c = extended
4.   **CompetitionDistance**:distance in meters to the nearest competitor store
5.   **CompetitionOpenSinceMonth**: the approximate month of the time when the nearest competitor was opened
6.   **CompetitionOpenSinceYear**: the approximate year of the time when the nearest competitor was opened   
7.   **Promo2**: a continuing and consecutive promotion, e.g., a coupon based mailing campaign, for some stores: 0 = store is not participating, 1 = store is participating
8.   **Promo2SinceWeek**: the calendar week when the store started participating in Promo2
9.   **Promo2SinceYear**: the year when the store started participating in Promo2
10.   **PromoInterval** : the consecutive intervals in which Promo2 is restarted, naming the months the promotion is started anew.e.g.,"Feb,May,Aug,Nov" means each round of the coupon based mailing campaign starts in February, May, August, November of any given year for that store, as the coupons, mostly for a discount on certain products are usually valid for three months, and a new round of mail needs to be sent to customers just before those coupons have expired













# 'Train' and 'Test' Datasets' Columns and Description

  

1.   **Store**: the anonymised store number
2.   **DayOfWeek**: the day of the week: 1 = Monday, 2 = Tuesday, …
3.   **Date**: the given date
4.  **Sales** : the turnover on a given day*
5.  **Customers** : the number of customers on a given day*
6.   **Open**: an indicator for whether the store was open on that day: 0 = closed, 1 = open
7.   **Promo**: indicates whether a store is running a store-specific
promo on that day
8.   **StateHoliday**: indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends.
      a = public holiday,
      b = Easter holiday,
      c = Christmas,
      0 = none
9.   **SchoolHoliday**: indicates if the (Store, Date) was affected by the closure of public schools

*test.csv is similar with train.csv except that Sales and Customers are unknown for the period of 01/08/2015 to 17/09/2015. *italicized text*














# Objective

Forecast sales for the period of August 1st to September 17th, 2015, using historical data from January 1st, 2013, to July 31st, 2015 by employing the most appropriate EDA technique, data preprocessing, suitable ML model, and evaluate performance metrics for accurate sales predictions.

# Datasets Loading

## Data Loading 'Store.csv'

In [ ]:
store = pd.read_csv('store.csv')

In [ ]:
print('Total row of store_df: ',len(store))

In [ ]:
print('Any NA at df_store: ')
print(store.isna().any())

In [ ]:
store.isnull().sum()

## Data Loading 'train.csv'

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
print('Total row of train: ',len(train))

In [ ]:
train.head()

In [ ]:
# Assuming 'Date' column is not in datetime format, convert it first
train['Date'] = pd.to_datetime(train['Date'], format='%d/%m/%Y')

subset = train[(train['Date'] >= '2015-01-08') & (train['Date'] <= '2015-01-12')]

# Display or use the filtered DataFrame as needed
subset

In [ ]:
unique_store_count = train['Store'].nunique()

print("Number of unique values in the 'Store' column:", unique_store_count)

In [ ]:
print('Any NA at train_df: ')
print(train.isna().any())

## Data Loading 'test.csv'

In [ ]:
test_df = pd.read_csv('test.csv')

In [ ]:
print('Total row of test_df: ',len(test_df))

In [ ]:
print('Any NA at test_df: ')
print(test_df.isna().any())

In [ ]:
test_df

In [ ]:
start_date = '13/08/2015'
end_date = '14/08/2015'

# Assuming 'Date' is the name of your date column
selected_rows = test_df[(test_df['Date'] >= start_date) & (test_df['Date'] <= end_date)]

selected_rows


# 2. Exploratory Data Analysis

## 2.1 Data Cleaning & Preparation

### 2.1.1 Data Imputation (store.csv)

#### Missing value on Promo2SinceWeek,Promo2SinceYear, PromoInterval columns needed a reasonable approach to impute it. However, there are no additional knowledge or context available to impute those values accurately. Hence, we prefer to fill it with 0 implying that they did not follow any consecutive promotion.

In [ ]:
promotion_to_fill_with_0 = ['Promo2SinceWeek','Promo2SinceYear','PromoInterval']

# Fill missing values in specified columns with 0
store[promotion_to_fill_with_0] = store[promotion_to_fill_with_0].fillna(0)


#### Then, missing value on CompetitionDistance, CompetitionOpenSinceMonth, and CompetitionOpenSinceYear can be imputed as it can be interpreted based on the their on patterns as follow.

####CompetitionDistance: impute by the median distance as the dataset is skewed (not normally distributed),CompetitionOpenSinceMonth: impute by the most frequent month competitor opened their store,CompetitionOpenSinceYear: impute by the most frequent year competitor opened their store

In [ ]:
competition_distance = store['CompetitionDistance']

plt.hist(competition_distance, bins='auto', color='blue', alpha=0.7, rwidth=0.85)
plt.title('Competition Distance Histogram')
plt.xlabel('Competition Distance')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
# Impute CompetitionDistance by the median distance
avg_distance = store['CompetitionDistance'].mean()
store['CompetitionDistance'].fillna(avg_distance, inplace=True)

# Impute CompetitionOpenSinceMonth by the most frequent month
most_freq_month = store['CompetitionOpenSinceMonth'].mode()[0]
store['CompetitionOpenSinceMonth'].fillna(most_freq_month, inplace=True)

# Impute CompetitionOpenSinceYear by the most frequent year
most_freq_year = store['CompetitionOpenSinceYear'].mode()[0]
store['CompetitionOpenSinceYear'].fillna(most_freq_year, inplace=True)

In [ ]:
print('Any NA at store_df: ',store.isna().any())

### 2.1.2 Data Merge 'store.csv' and 'train.csv'

In [ ]:
# Merging 'store_df' with 'train_df' based on the 'Store' column as merged_df
merged_df = pd.merge(train, store, on='Store', how='left')

In [ ]:
merged_df.head()

In [ ]:
print('Total row of merged_df: ',len(merged_df))

In [ ]:
# Change 'date' column type to datetime
merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='%d/%m/%Y')
merged_df.info()

In [ ]:
latest_date_merged = merged_df['Date'].max()
print("Latest Date in 'Date' Column:", latest_date_merged)


In [ ]:
# Get the count of unique values in each column
unique_value_counts = merged_df.nunique()

# Display the count of unique values in each column
print("Count of unique values in each column:")
print(unique_value_counts)

In [ ]:
# Assuming 'Date' is a column in your DataFrame
date_cutoff = '2015-07-31'

# Convert 'Date' column to datetime if necessary
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Filter rows in the DataFrame based on the 'Date' column
merged_df = merged_df[merged_df['Date'] <= date_cutoff]




In [ ]:
merged_result_agg = merged_df.set_index('Date').resample('D')['Sales'].mean().reset_index()
fig_merged = px.line(merged_result_agg, x='Date', y='Sales', title='Actual Sales on Merged dataset')
fig_merged.show()

In [ ]:
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Assuming sales_a, sales_b, sales_c, sales_d are your time series data for different categories

# Perform seasonal decomposition
decomposition_a = seasonal_decompose(sales_a, model='additive', period=365)
decomposition_b = seasonal_decompose(sales_b, model='additive', period=365)
decomposition_c = seasonal_decompose(sales_c, model='additive', period=365)
decomposition_d = seasonal_decompose(sales_d, model='additive', period=150)

# Create subplots in a 2x2 grid
fig = make_subplots(rows=2, cols=2, subplot_titles=['Sales Store Type A', 'Sales Store Type B', 'Sales Store Type C', 'Sales Store Type D'],
                    shared_xaxes=True, vertical_spacing=0.1)

# Add traces for trend to each subplot
fig.add_trace(go.Scatter(x=decomposition_a.trend.index, y=decomposition_a.trend, mode='lines', name='Trend A'), row=1, col=1)
fig.add_trace(go.Scatter(x=decomposition_b.trend.index, y=decomposition_b.trend, mode='lines', name='Trend B'), row=1, col=2)
fig.add_trace(go.Scatter(x=decomposition_c.trend.index, y=decomposition_c.trend, mode='lines', name='Trend C'), row=2, col=1)
fig.add_trace(go.Scatter(x=decomposition_d.trend.index, y=decomposition_d.trend, mode='lines', name='Trend D'), row=2, col=2)

# Update layout
fig.update_layout(height=600, width=1400, title_text='Trend of Sales in Different Categories', showlegend=True)

# Add common x-axis title using annotation
fig.add_annotation(text="Date", xref="paper", yref="paper", x=0.5, y=-0.1, showarrow=False)

# Show the figure
fig.show()


### 2.1.3 Checking for Missing Values of the the final train set (merged)

In [ ]:
print('Any NA at merged_df: ',merged_df.isna().any())

### 2.1.4 Summary and detail of the Train set

In [ ]:
print('Number of training rows:', merged_df.index.nunique())
print('Number of test rows:', test_df.index.nunique())

In [ ]:
merged_df.describe()

In [ ]:
merged_df.head(6)

In [ ]:
latest_date_merged = merged_df['Date'].max()
print("Latest Date in 'Date' Column:", latest_date_merged)


In [ ]:
merged_df.info()

## 2.2 Exploratory Data Analysis (EDA)

### 2.2.1 Target Variable: Sales

#### Normality Checking

First, I check the normality for the relationship between Assorment VS Sales and Store VS Sales for several purpose: distribution understanding, statistical assumption, identifying outliers, and decision support.

In [ ]:
# Assuming 'Assortment' and 'Sales' are columns in merged_df
unique_Assortment = merged_df['Assortment'].unique()

# Filter the data for the selected Assortment
subset_df = merged_df[merged_df['Assortment'].isin(unique_Assortment)]

# Checking normality using box plots
plt.figure(figsize=(5, 5))
plt.ylim(0, 25000)
dep_boxplot = sns.boxplot(x="Assortment", y="Sales", data=subset_df)
plt.show()


The distributions of sales for assortments ‘a’ and ‘c’ appear to be skewed towards lower values, and therefore, they do not seem to be normally distributed. The distribution of sales for assortment ‘b’ appears more balanced, but the presence of an outlier could affect its normality.

In [ ]:
# Assuming 'Store' and 'Sales' are columns in merged_df
unique_stores = merged_df['Store'].unique()

# Select a random subset of 20 stores
random_stores = random.sample(list(unique_stores), 100)

# Filter the data for the selected stores
subset_df = merged_df[merged_df['Store'].isin(random_stores)]

# Checking normality using box plots
plt.figure(figsize=(40, 10))
plt.ylim(0, 7000)
dep_boxplot = sns.boxplot(x="Store", y="Sales", data=subset_df)
plt.show()


Each store's sales differs in terms of range & outlier presence. it appears that the distribution of sales across different stores does not seem to be normal. There are noticeable variations in sales among different stores; some have very high sales while others have low. This suggests that there could be potential outliers where certain stores have significantly higher or lower sales than others.

#### Analysis on Total Sales

#### Monthly Average Sales

In [ ]:
# Monthly Average Sales

# Set month
train_time = merged_df.set_index('Date').resample('M').Sales.mean()
train_time = pd.DataFrame(train_time)
train_time['year'] = train_time.index.year


fig = px.line(train_time, x=train_time.index, y='Sales', color='year', title='Monthly Average Sales')
fig.update_layout(width=900, height=500)

In [ ]:
# Monthly Total Sales

# Set month
train_time = merged_df.set_index('Date').resample('M').Sales.sum()
train_time = pd.DataFrame(train_time)
train_time['year'] = train_time.index.year

fig = px.line(train_time, x=train_time.index, y='Sales', color='year', title='Monthly Total Sales')
fig.update_layout(width=900, height=500)

# If you want to show the x-axis as months instead of the full date
fig.update_layout(xaxis=dict(type='category'))


The monthly average of daily sales of all stores increased every year, with the trends always rising and hit the highest peak every December. The sales generally started plumetting in August and back rising in early September each year—a fact that may need to be considered later during the modelling process when choosing time lags.



#### Annual Average Sales

In [ ]:
# Annual Average Sales
train_time_year = merged_df.set_index('Date').resample('Y').Sales.mean()
train_time_year = pd.DataFrame(train_time_year)
train_time_year['year'] = train_time_year.index.year
fig = px.line(train_time_year, x=train_time_year.index, y='Sales', title = 'Annual Average Sales')
fig.update_layout(width=900, height=500)

#### Day of Week Average Sales

In [ ]:
merged_day = merged_df.copy()
merged_day['year'] = merged_df.Date.dt.year #add year column
merged_day['DayOfWeek'] = merged_df.Date.dt.dayofweek+1 #add day of week column
day_gb = merged_day.groupby(['year', 'DayOfWeek'])['Sales'].mean().reset_index()
px.line(day_gb, x='DayOfWeek', y='Sales', color='year', title = 'Day of Week Average Sales')

In [ ]:
import plotly.express as px

# Copy the DataFrame
merged_day = merged_df.copy()

# Add year column
merged_day['year'] = merged_df.Date.dt.year

# Add day of week column
merged_day['DayOfWeek'] = merged_df.Date.dt.dayofweek + 1

# Group by year, day of week, and Promo
day_gb = merged_day.groupby(['year', 'DayOfWeek', 'Promo'])['Sales'].mean().reset_index()

# Create separate figures based on the 'Promo' column
fig_promo_0 = px.line(day_gb[day_gb['Promo'] == 0], x='DayOfWeek', y='Sales', color='year',
                      title='Day of Week Average Sales (Promo 0)')

fig_promo_1 = px.line(day_gb[day_gb['Promo'] == 1], x='DayOfWeek', y='Sales', color='year',
                      title='Day of Week Average Sales (Promo 1)')

# Show the figures
fig_promo_0.show()
fig_promo_1.show()


Looking at average sales on specific days of the week, their lowest points happened in weekend (day 7), before ascending back on every first day of the week. However, we may also observed that the annual sales and day-of-week have distinct sales pattern.

#### Open Days

In [ ]:
# Open Days: Is there any day where the stores close?
merged_df_time = merged_df[['Date', 'Sales']].groupby('Date').mean()
merged_df_time['open'] = 1
full_range = pd.date_range(start=merged_df_time.index.min(), end=merged_df_time.index.max(), freq='D')
full_df = pd.DataFrame(index=full_range)

full_df = full_df.merge(merged_df_time['open'], left_index=True, right_index=True, how='left').fillna(0)

fig = px.line(full_df, x=full_df.index, y='open', title='Open Days')
fig.update_layout(width=700, height=700)

From Figure above, the sales accross all stores are shown to be always ongoing throughout the year. Hence, suggesting despite the SchoolHoliday / StateHoliday variable, not all the stores were closed.

In [ ]:
print('The store holidays are on', pd.date_range(start=merged_df_time.index.min(), end=merged_df_time.index.max(), freq='D').difference(merged_df_time.index).values)

In [ ]:
stores_with_zero_sales = merged_df[merged_df['Sales'] == 0]
print("Rows with Sales equal to 0:")
stores_with_zero_sales

In [ ]:
# Calculate average total number of days with 'Sales' equal to 0 for each store
average_days_with_zero_sales = stores_with_zero_sales.groupby('Store')['Date'].count().mean() / 2.5

print("Average total days per-year with Sales equal to 0 for each Store:", average_days_with_zero_sales)

However, it doesn't mean that *each* store always has sales, in a year the store roughly were having 62 days that having 0 sales. These date and store who were having 0 sales doesn't have any pattern, mean it can be considered as an outliers

#### Stores with 0 Sales

In [ ]:
# Count occurrences of each store
store_counts = stores_with_zero_sales['Store'].value_counts()

# Identify potential outliers in occurrences using IQR
Q1 = store_counts.quantile(0.25)
Q3 = store_counts.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

potential_outliers = store_counts[(store_counts < lower_bound) | (store_counts > upper_bound)]

# Plot box plot to visualize occurrences distribution and potential outliers
plt.figure(figsize=(3, 5))
sns.boxplot(store_counts)
plt.title('Box Plot of Count of Store with 0 Sales Occurrences')
plt.xlabel('Occurrences')
plt.ylabel('Count of Days with 0 Sales')
plt.show()

In [ ]:
# Select stores with more than 180 occurrences of Sales equal to 0
selected_stores = store_counts[store_counts > 180]

# Print the result
print("Stores with more than 180 occurrences of Sales equal to 0:")
print(selected_stores)

# Plot occurrences using a bar plot
selected_stores.plot(kind='bar', rot=0, color='skyblue')
plt.title('Stores with more than 180 occurrences of Sales equal to 0 throughout 2013-Jun2015')
plt.xlabel('Store')
plt.ylabel('Count of Days 0 Sales')
plt.show()

18 Stores observed having occurences 0 sales more than 180 days throughout the dataset (January 2013 - 31 July 2015), we will later consider these Store to be excluded for the next phase, as it shows an anomaly behavior and affecting data quality for model generalization later.

In [ ]:
# Define a custom color palette
custom_palette = {'a': 'blue', 'b': 'green', 'c': 'orange', 'd': 'red'}

# Set the order of the columns based on StoreType
store_type_order = ['a', 'b', 'c', 'd']

# Plot
sns.catplot(data=train_store, x='Month', y="Sales",
            col='StoreType',  # per store type in cols
            hue='StoreType',
            row='Promo',  # per promo in the store in rows
            kind='point',
            palette=custom_palette,
            col_order=store_type_order)

# Customize the layout
plt.subplots_adjust(top=0.9)
plt.suptitle('Sales Based on StoreType and Promo')
plt.show()


In [ ]:
# Compute the correlation matrix
# exclude 'Open' variable
corr_all = train_store.drop('Open', axis = 1).corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_all, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_all, mask = mask,
            square = True, linewidths = .5, ax = ax, cmap = "coolwarm")
plt.show()

In [ ]:
# sale per customer trends
sns.catplot(data = train_store, x = 'DayOfWeek', y = "Sales",
               col = 'Promo',
               row = 'Promo2',
               hue = 'Promo2',
               palette = 'RdPu',
            kind = 'point')

## 2.2.2 Univariate & Multivariate Analysis

### Store Performance Analysis

In [ ]:
#store's daily sales performance
merged_df['Store'] = merged_df['Store'].astype('category')
train_store = merged_df.groupby(['Date', 'Store']).Sales.mean().reset_index()


fig = px.line(train_store, x='Date', y='Sales', color='Store', title='Daily Sales Average by Store Number')
fig.update_layout(width=1500, height=700)

In [ ]:
#boxplot
px.box(train_store, x='Store', y='Sales', color='Store', title='Average Sales by Store Number')

### 'Customer' Analysis

'Customer' must be one variable that having positive correlation with Sales.

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(x='Customers', y='Sales', data=merged_df, scatter_kws={'alpha':0.3}, line_kws={'color': 'red'})
plt.title('Scatter Plot with Regression Line: Sales vs Number of Customers')
plt.xlabel('Number of Customers')
plt.ylabel('Sales')
plt.show()

### 'Promo' Analysis

'Promo' become one of the variable is commonly suspected having high of influence towards the Sales. Hence, analyze this variable first will be reasonable

In [ ]:
import plotly.express as px

# Assuming 'Date' is already in datetime format
special_offer = merged_df.set_index('Date').resample('M')['Promo', 'Sales'].mean().reset_index()

# Create a figure with bar plot for 'Promo' and line plot for 'Sales'
fig = px.bar(special_offer, x='Date', y='Promo', title='Promo and Sales Trend')
fig.add_trace(px.line(special_offer, x='Date', y='Sales').update_traces(yaxis='y2', line=dict(color='blue')).data[0])

# Set y-axis titles
fig.update_layout(yaxis_title='Promo', width=1000, height=500)

# Add a second y-axis for 'Sales'
fig.update_layout(
    yaxis2=dict(
        overlaying='y',
        side='right',
        title='Sales'
    )
)

# Show the plot
fig.show()


From figure above, we might observed that the Promo has correllation with the Sales that we will include in our feature engineering and modeling stage later , while also need to be noted that some of the trend that not relevant that may be caused Promo Interval variable.

### 'Competition Distance' Analysis

Next, Competition Distance become the next variable will be discussed as hypotetically having impact to the Sales.

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(x='CompetitionDistance', y='Sales', data=merged_df, scatter_kws={'alpha':0.3}, line_kws={'color': 'red'})
plt.title('Scatter Plot with Regression Line: Sales vs. Competition Distance')
plt.xlabel('Competition Distance')
plt.ylabel('Sales')
plt.show()

From figure above, turns out Competition Distance is relatively having negative correllation with the Sales as most of the highest sales is produced when having a near competitor.

### 'Assortment' Analysis

In [ ]:
#product's daily sales performance
merged_prod = merged_df.groupby(['Date', 'Assortment']).Sales.mean().reset_index()
fig = px.line(merged_prod, x='Date', y='Sales', color='Assortment', title='Sales by Assortment Type')
fig.update_layout(width=1000, height=400)

In [ ]:
#boxplot
merged_prod = merged_df.groupby(['Date', 'Assortment']).Sales.mean().reset_index()
fig = px.box(merged_prod, x='Assortment', y='Sales', color='Assortment', title='Average Sales by Assortment Type')
fig.update_layout(width=700, height=500)

### 'Store Type' Analysis

In [ ]:
#product's daily sales performance
merged_prod = merged_df.groupby(['Date', 'StoreType']).Sales.mean().reset_index()
fig = px.line(merged_prod, x='Date', y='Sales', color='StoreType', title='Sales by Store Type')
fig.update_layout(width=1000, height=400)

In [ ]:
#boxplot
merged_prod = merged_df.groupby(['Date', 'StoreType']).Sales.mean().reset_index()
fig = px.box(merged_prod, x='StoreType', y='Sales', color='StoreType', title='Average Sales by Store Type')
fig.update_layout(width=700, height=600)

In [ ]:
# opened stores with zero sales
zero_sales = train[(train.Open != 0) & (train.Sales == 0)]
print("In total: ", zero_sales.shape)
zero_sales.head(5)

## Conclusion from EDA

1. Annual sales and day-of-week sales have distinct sales patterns.
2. Despite having state and school Holiday, generally the ROSSMAN through its store accross the country is always having a sales activity everyday.
3. Promo has positive correlation to sales.
4. Competition Distance has negative correlation to sales.
5. Some stores, have only being consistent generating sales started around 8 October 2014. This strengthens the argument of
using only 8 October 2014 onward data for the modeling stage.
6. Assortment type C relatively do not have any pattern as the sales very fluctuative throughout the years.

# 3. Data Pre-Processing

## 3.1 Delete unnecessary records

The EDA revealed the issue of some variables that having unneccessary records and need to be deleted

- Closed stores which has 0 sales occurences

In [ ]:
# Remove rows with 'Store' values in selected_stores.index
filtered_df = merged_df[(merged_df["Open"] != 0) & (merged_df['Sales'] != 0)]


In [ ]:
Overview = merged_df.set_index('Date').resample('D')['Sales'].mean().reset_index()
aa_merged = px.line(Overview, x='Date', y='Sales', title='Actual Sales on Merged dataset')
aa_merged.show()

## 3.2 Feature Engineering (Time-series feature)

### 3.2.2 Adding Sales Lag

Adding sales lag

In [ ]:
prod_info_sorted = filtered_df.sort_values(by=['Store', 'StoreType','Assortment','Date'])

prod_info_sorted['sales_lag7'] = prod_info_sorted.groupby(['Store', 'StoreType','Assortment'])['Sales'].shift(7) #lag7
prod_info_sorted['sales_lag12'] = prod_info_sorted.groupby(['Store', 'StoreType','Assortment'])['Sales'].shift(12) #lag14
prod_info_sorted['sales_lag14'] = prod_info_sorted.groupby(['Store', 'StoreType','Assortment'])['Sales'].shift(14) #lag21
prod_info_sorted['sales_lag24'] = prod_info_sorted.groupby(['Store', 'StoreType','Assortment'])['Sales'].shift(24) #lag90


In [ ]:
prod_info_sorted

## 3.3 Encode

In [ ]:
from sklearn.preprocessing import LabelEncoder

prod_info_encoded = prod_info_sorted.copy()
label_encoder = LabelEncoder()

# Encode 'Assortment'
assortment_values = prod_info_encoded['Assortment'].astype(str)
label_encoder.fit(assortment_values)
prod_info_encoded['encoded_Assortment'] = label_encoder.transform(assortment_values)

# Encode 'StoreType'
store_type_values = prod_info_encoded['StoreType'].astype(str)
label_encoder.fit(store_type_values)
prod_info_encoded['encoded_StoreType'] = label_encoder.transform(store_type_values)

# Encode 'PromoInterval'
promo_interval_values = prod_info_encoded['PromoInterval'].astype(str)
label_encoder.fit(promo_interval_values)
prod_info_encoded['encoded_PromoInterval'] = label_encoder.transform(promo_interval_values)

prod_info_encoded = prod_info_encoded.sort_values(by=['Date', 'Store', 'Assortment']).reset_index(drop=True)

In [ ]:
prod_info_encoded.head(5)

In [ ]:
start_date = '2015-07-30'
end_date = '2015-07-31'

# Assuming 'Date' is the name of your date column
selected_rows = prod_info_encoded[(prod_info_encoded['Date'] >= start_date) & (prod_info_encoded['Date'] <= end_date)]

selected_rows.head(5)

In [ ]:
prod_info_encoded.isna().any()

## 3.5 Split into train and test(validation) dataset

In [ ]:

# Calculate the index to split the DataFrame
split_index = int(len(prod_info_encoded) * 0.8)

# Create training set (80%)
train_preprocessed = prod_info_encoded.iloc[:split_index]

# Create testing set (20%)
# Note: Now, the testing set only includes rows with 'Date' until 2015-07-31
test_preprocessed = prod_info_encoded.iloc[split_index:]

# Print the shapes of the sets to verify the split
print("Train shape:", train_preprocessed.shape)
print("Test shape:", test_preprocessed.shape)


In [ ]:
# For train_preprocessed
train_result_agg = train_preprocessed.set_index('Date').resample('D')['Sales'].mean().reset_index()
fig_train = px.line(train_result_agg, x='Date', y='Sales', title='Actual Sales on Train dataset')
fig_train.show()

# For test_preprocessed
test_result_agg = test_preprocessed.set_index('Date').resample('D')['Sales'].mean().reset_index()
fig_test = px.line(test_result_agg, x='Date', y='Sales', title='Actual Sales on Test dataset')
fig_test.show()

## 3.6 Export merged_df into CSV

In [ ]:
from google.colab import files
train_preprocessed.to_csv('train_preprocessed', index=False)
files.download('train_preprocessed')

test_preprocessed.to_csv('test_preprocessed', index=False)
files.download('test_preprocessed')

# Extend the test.csv

In [ ]:
test_df['Date'] = test_df['Date'].str.strip()
test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d/%m/%Y', errors='coerce')


Check if the following date can be converted to datetime correctly after stripping the whitespace

In [ ]:
start_date = '2015-08-10'
end_date = '2015-08-12'
selected_rows = test_df[(test_df['Date'] >= start_date) & (test_df['Date'] <= end_date)]
print(selected_rows)


In [ ]:
# Convert 'Date' column to datetime format in merged_df
merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')

# Convert 'Date' column to datetime format in test_df
test_df['Date'] = pd.to_datetime(test_df['Date'], errors='coerce')

# Concatenate the two DataFrames
extended_df = pd.concat([merged_df, test_df], ignore_index=True)

# Sort the DataFrame by 'Date' in descending order
extended_df = extended_df.sort_values(by='Date', ascending=False)

# Display the sorted and extended DataFrame
extended_df

In [ ]:
start_date = '2015-09-01'
end_date = '2015-09-07'

# Assuming 'Date' is the name of your date column
selected_rows = extended_df[(extended_df['Date'] >= start_date) & (extended_df['Date'] <= end_date)]

selected_rows

In [ ]:
extended_sorted = extended_df.fillna(0).sort_values(by=['Store', 'DayOfWeek', 'Date'])

extended_sorted['sales_lag7'] = extended_sorted.groupby(['Store', 'DayOfWeek'])['Sales'].shift(7) #lag7
extended_sorted['sales_lag14'] = extended_sorted.groupby(['Store', 'DayOfWeek'])['Sales'].shift(14) #lag14
extended_sorted['sales_lag12'] = extended_sorted.groupby(['Store', 'DayOfWeek'])['Sales'].shift(12) #lag12
extended_sorted['sales_lag24'] = extended_sorted.groupby(['Store', 'DayOfWeek'])['Sales'].shift(24) #lag24



In [ ]:
filtered_df = extended_sorted[(extended_sorted['Date'] >= '2015-08-01') & (extended_sorted['Date'] <= '2015-09-17')]
sorted_df = filtered_df.sort_values(by='Date', ascending=True)
sorted_df

In [ ]:
extended_sorted = extended_sorted[(extended_sorted['Date'] >= '2015-08-01') & (extended_sorted['Date'] <= '2015-09-17')]


In [ ]:
from google.colab import files
extended_sorted.to_csv('forecasting_df', index=False)
files.download('forecasting_df')